<a href="https://colab.research.google.com/github/AdityaVardhanGara/Colab/blob/master/BPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd

#reading .txt file
text = pd.read_csv("sample.txt",header=None)

#converting a dataframe into a single list 
corpus=[]
for row in text.values:
    tokens = row[0].split(" ")
    for token in tokens:
        corpus.append(token)

In [17]:
vocab = list(set(" ".join(corpus)))
vocab.remove(' ')

#split the word into characters
corpus = [" ".join(token) for token in corpus]

#appending </w>
#corpus=[token+' </w>' for token in corpus]

In [18]:
import collections

#returns frequency of each word
corpus = collections.Counter(corpus)

#convert counter object to dictionary
corpus = dict(corpus)
print("Corpus:",corpus)

Corpus: {'l o w': 5, 'l o w e r': 2, '': 1, 'w i d e s t': 3, 'n e w e s t': 6}


In [19]:
def get_stats(corpus):
    pairs = collections.defaultdict(int)
    for word, freq in corpus.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

In [20]:
import re
def merge_vocab(pair, corpus_in):
    corpus_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in corpus_in:
        w_out = p.sub(''.join(pair), word)
        corpus_out[w_out] = corpus_in[word]
    
    return corpus_out

In [21]:
pairs = get_stats(corpus)
print(pairs)

defaultdict(<class 'int'>, {('l', 'o'): 7, ('o', 'w'): 7, ('w', 'e'): 8, ('e', 'r'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3, ('e', 's'): 9, ('s', 't'): 9, ('n', 'e'): 6, ('e', 'w'): 6})


In [22]:
best = max(pairs, key=pairs.get)
print("Most Frequent pair:",best)

Most Frequent pair: ('e', 's')


In [23]:
corpus = merge_vocab(best, corpus)
print("After Merging:", corpus)

#convert a tuple to a string
best = "".join(list(best))

#append to merge list and vocabulary
merges = []
merges.append(best)
vocab.append(best)

After Merging: {'l o w': 5, 'l o w e r': 2, '': 1, 'w i d es t': 3, 'n e w es t': 6}


In [24]:
num_merges = 10
for i in range(num_merges):
    
    #compute frequency of bigrams in a corpus
    pairs = get_stats(corpus)
    
    #compute the best pair
    best = max(pairs, key=pairs.get)
    
    #merge the frequent pair in corpus
    corpus = merge_vocab(best, corpus)
    
    #append to merge list and vocabulary
    merges.append(best)
    vocab.append(best)

#convert a tuple to a string
merges_in_string = ["".join(list(i)) for i in merges]
print("BPE Merge Operations:",merges_in_string)

BPE Merge Operations: ['es', 'est', 'lo', 'low', 'ne', 'new', 'newest', 'wi', 'wid', 'widest', 'lowe']


In [25]:
oov ='lowest'

#tokenize OOV into characters
oov = " ".join(list(oov))

#append </w> 
oov = oov + ' </w>'

#create a dictionary
oov = { oov : 1}

In [26]:
i=0
while(True):

    #compute frequency
    pairs = get_stats(oov)

    #extract keys
    pairs = pairs.keys()
    
    #find the pairs available in the learned operations
    ind=[merges.index(i) for i in pairs if i in merges]

    if(len(ind)==0):
        print("\nBPE Completed...")
        break
    
    #choose the most frequent learned operation
    best = merges[min(ind)]
    
    #merge the best pair
    oov = merge_vocab(best, oov)
    
    print("Iteration ",i+1, list(oov.keys())[0])
    i=i+1

Iteration  1 lo w e s t </w>
Iteration  2 low e s t </w>
Iteration  3 lowe s t </w>

BPE Completed...


# New Section